# Split Data

In [11]:
import numpy as np
import os
from tqdm import tqdm
import pandas as pd

In [ ]:
data_group = pd.read_csv('data_group.csv')
data_group

In [3]:
original_data_path = './data/original_data'
new_data_path = './data/new_data'
limit = 295000

In [ ]:
# group the data by group id
groups = data_group.groupby("group")

# go through each group
for group_id, group in tqdm(groups):
    
    # summary the length of the original data and split data
    sum_group_len = group.num_data.sum()
    split_data_len = 0
    
    # go through each file in the group
    for j in range(len(group)):
        split_file_len = 0
        file_name = group.file_name.values[j]
        file_len = group.num_data.values[j]
        need_key_number = int(np.ceil(file_len / limit))
        data = pd.read_stata(os.path.join(original_data_path, file_name))
        
        # split each file
        # rename the new file in the form like (groupId_fileName_numOfSplit)
        for i in range(need_key_number):
            new_file_name = "{}_{}_{}.csv".format(group_id, file_name.split('.')[0], i)
            sub_data = data.loc[i*limit: (i+1)*limit-1]
            split_file_len += len(sub_data)
            split_data_len += len(sub_data)
            sub_data.to_csv(os.path.join(new_data_path, new_file_name), index=False)
            
        assert split_file_len == file_len
        
    assert split_data_len == sum_group_len